In [1]:
import scipy.stats as ss
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

##### 1 task

In [3]:
from numpy.linalg import matrix_power

def beta_cov(x, y):
    n = x.shape[0]
    beta_star = np.matmul(matrix_power((1/n)*np.matmul(x.T, x), -1),  (1/n)*np.matmul(x.T, y))
    H = (1/n)*np.matmul(x.T, x)
    # print(beta_star.shape)
    # print(x.shape)
    # print(y.shape)
    # print(x.T.shape)
    # print((y - np.matmul(x, beta_star)).shape)
    D = np.diag((y - np.matmul(x, beta_star))[:,0])**2
    # print(D.shape)
    V = np.matmul(x.T, np.matmul(D, x))/n
    # print("V:", V.shape)
    # print(H.shape)
    H_ = matrix_power(H, -1)
    sigma = np.matmul(H_, np.matmul(V, H_))
    return beta_star, sigma


rng = np.random.default_rng(1337)
x = ss.norm.rvs(size=(100, 3), random_state=rng)
eps = ss.norm.rvs(size=(100, 1), random_state=rng)
beta = ss.norm.rvs(size=(3, 1), random_state=rng)
y = x @ beta + eps
x.shape[0]

print(*beta_cov(x, y), sep='\n')

[[-1.0878702 ]
 [-0.5688717 ]
 [ 0.93436482]]
[[0.67038035 0.0662398  0.05856229]
 [0.0662398  0.60565119 0.0659719 ]
 [0.05856229 0.0659719  1.91253231]]


##### 2 task

In [121]:
from numpy.linalg import matrix_power

def beta_cov(x, y):
    n = x.shape[0]
    beta_star = np.matmul(matrix_power(
        (1/n)*np.matmul(x.T, x), -1),  (1/n)*np.matmul(x.T, y))
    H = (1/n)*np.matmul(x.T, x)
    D = np.diag((y - np.matmul(x, beta_star))[:, 0])**2
    V = np.matmul(x.T, np.matmul(D, x))/n
    H_ = matrix_power(H, -1)
    sigma = np.matmul(H_, np.matmul(V, H_))
    return beta_star, sigma


def interval(x, y, c):
    beta_star, sigma_star = beta_cov(x, y)
    sigma_square = np.matmul(c, np.matmul(sigma_star, c.T))
    ci = ss.norm.interval(0.95, loc=0, scale = np.sqrt(sigma_square)/np.sqrt(len(y)))
    ci = [ci[0][0][0], ci[1][0][0]] 
    conf_interval = c@beta_star + ci
    return conf_interval[0]


data = pd.read_csv(
    'https://stepik.org/media/attachments/lesson/832665/Fish.csv')
data = data.drop(40, axis=0)

x = data[['Length1', 'Height', 'Width']].apply(np.log)
x['const'] = 1
y = data[['Weight']].apply(np.log)
c = np.array([[1, 1, 1, 0]])  # в таком порядке, так как константа последняя

print(interval(x.values, y.values, c))

[2.96242792 3.05267914]


##### 3 task

In [84]:
def beta_WLS(x, y, w):
    W = np.diag(w(x))
    beta = np.linalg.solve(np.dot(np.dot(x.T, W), x), np.dot(np.dot(x.T, W), y))
    return beta


rng = np.random.default_rng(0)
x = ss.norm.rvs(size=(100, 3), random_state=rng)
eps = ss.norm.rvs(size=(100, 1), random_state=rng)
beta = ss.norm.rvs(size=(3, 1), random_state=rng)
y = x @ beta + eps * np.abs(x[:, [0]])
def w(x): return np.abs(x[:, 0])


print(beta_WLS(x, y, w))

[[-0.19503921]
 [ 0.5637669 ]
 [-2.00190378]]


##### 4 task

In [118]:
def beta_WLS(x, y, w):
    W = np.diag(w(x))
    beta = np.linalg.solve(np.dot(np.dot(x.T, W), x), np.dot(np.dot(x.T, W), y))
    return beta

def cov_WLS(x, y, w):
    beta_star = beta_WLS(x, y, w)
    W = np.diag(w(x))
    D = np.diag((y -  x @ beta_star)[:, 0])**2
    V =  x.T @ W @ W @ D @ x / len(y)
    H = (1 / len(y)) * x.T @ W @ x
    sigma = np.linalg.inv(H) @ V @ np.linalg.inv(H)
    return sigma

rng = np.random.default_rng(0)
x = ss.norm.rvs(size=(100, 3), random_state=rng)
eps = ss.norm.rvs(size=(100, 1), random_state=rng)
beta = ss.norm.rvs(size=(3, 1), random_state=rng)
y = x @ beta + eps * np.abs(x[:, [0]])
def w(x): return np.abs(x[:, 0])


print(cov_WLS(x, y, w))

[[ 2.34752824  0.0547367  -2.01159742]
 [ 0.0547367   2.16796233 -0.09831566]
 [-2.01159742 -0.09831566  5.3083249 ]]


##### 5 task

In [123]:
def beta_WLS(x, y, w):
    W = np.diag(w(x))
    beta = np.linalg.solve(np.dot(np.dot(x.T, W), x), np.dot(np.dot(x.T, W), y))
    return beta

def cov_WLS(x, y, w):
    beta_star = beta_WLS(x, y, w)
    W = np.diag(w(x))
    D = np.diag((y -  x @ beta_star)[:, 0])**2
    V =  x.T @ W @ W @ D @ x / len(y)
    H = (1 / len(y)) * x.T @ W @ x
    sigma = np.linalg.inv(H) @ V @ np.linalg.inv(H)
    return sigma

def interval(x, y, w, c):
    beta_star = beta_WLS(x, y, w)
    sigma_star = cov_WLS(x, y, w)
    sigma_square = np.matmul(c, np.matmul(sigma_star, c.T))
    ci = ss.norm.interval(0.95, loc=0, scale = np.sqrt(sigma_square)/np.sqrt(len(y)))
    ci = [ci[0][0][0], ci[1][0][0]] 
    conf_interval = c@beta_star + ci
    return conf_interval[0]

data['Volume'] = data['Length1'] * data['Height'] * data['Width']
x = np.column_stack([np.ones(len(data)), data['Volume'].values])
y = data[['Weight']].values
c = np.array([[1, 0]])

def w(x): return 1 / x[:, 1] ** 2

print(interval(x, y, w, c))

def w(x): return np.ones(len(x))

print(interval(x, y, w, c))

[0.73977544 1.84962684]
[ 0.44463417 37.52791691]


##### 6 task

In [103]:
def psi(x, y, beta):
    mu = np.exp(np.matmul(x, beta))
    psi_t = (y - mu) * x
    return psi_t.T


rng = np.random.default_rng(0)
x = ss.norm.rvs(size=(1, 5), random_state=rng)
beta = ss.norm.rvs(size=(5, 1), random_state=rng)
y = ss.poisson(np.exp(x @ beta)).rvs(random_state=rng)

print(psi(x, y, beta))

[[-0.24589836]
 [ 0.25836564]
 [-1.25251412]
 [-0.20515964]
 [ 1.04764166]]


##### 7 task

In [125]:
import numpy as np

def psi(x, y, beta):
    """
    Вычисляет значение градиента логарифма плотности вероятности p(y|x,t) по t в точке t=beta.
    
    Параметры:
        x: numpy.ndarray, строка (вектор) размера 1 x k.
        y: float, число.
        beta: numpy.ndarray, вектор размера k x 1.
        
    Возвращает:
        psi_value: numpy.ndarray, градиент логарифма плотности вероятности в точке t=beta.
    """
    # Преобразование строк и чисел в нужный формат массивов
    x = np.array(x)
    beta = np.array(beta)
    
    # Вычисление mu (ожидаемого значения)
    mu = np.exp(x @ beta)
    
    # Вычисление производной логарифма плотности вероятности по t
    psi_value = (y - mu) * x
    
    return psi_value.reshape(beta.shape)

def cov_pois(x, y, beta):
    """
    Вычисляет оценку матрицы рассеивания beta.
    
    Параметры:
        x: numpy.ndarray, матрица размера n x k.
        y: numpy.ndarray, вектор размера n x 1.
        beta: numpy.ndarray, вектор размера k x 1.
        
    Возвращает:
        cov_beta: numpy.ndarray, оценка матрицы рассеивания beta.
    """
    n, k = x.shape
    
    # Инициализация матрицы рассеивания
    cov_beta = np.zeros((k, k))
    
    # Вычисление градиента логарифма плотности вероятности по t для каждого наблюдения
    psi_values = np.array([psi(x[i], y[i], beta) for i in range(n)])
    
    # Вычисление матрицы рассеивания
    for i in range(n):
        cov_beta += np.outer(psi_values[i], psi_values[i])
    
    cov_beta /= n
    
    return cov_beta

import numpy as np
import scipy.stats as ss
from scipy.optimize import minimize

rng = np.random.default_rng(0)
x = ss.norm.rvs(size=(100, 5), random_state=rng)
beta = ss.norm.rvs(size=(5, 1), random_state=rng)
y = ss.poisson(np.exp(x @ beta)).rvs(random_state=rng)

def mlogL(beta):
    return -ss.poisson.logpmf(y, mu=np.exp(x @ beta.reshape(-1, 1))).sum()

beta_star = minimize(mlogL, x0=np.zeros(x.shape[1])).x.reshape(-1, 1)

print(cov_pois(x, y, beta_star))


[[ 4.5917873   0.388386    0.9059114  -5.51186074  2.85805797]
 [ 0.388386    4.4738625   0.37613232 -2.86765339 -0.35458691]
 [ 0.9059114   0.37613232  3.1787806  -1.15306063 -0.98362199]
 [-5.51186074 -2.86765339 -1.15306063 16.510745   -3.78269525]
 [ 2.85805797 -0.35458691 -0.98362199 -3.78269525 10.31941884]]


##### 8 task

In [167]:
def check(phi_star, phi, sigma, n):
    value = ((phi_star - phi).T @  np.linalg.inv((sigma/n)) @ (phi_star - phi))[0][0]
    sample = ss.chi2.rvs(phi.shape[0], size = 10000)
    threshold = np.quantile(sample, 0.95)**2    
    return value <= threshold


# TESTING ZONE 
beta_star = np.array([[1.4295077 ], [0.62930259], [0.94874324]])
beta = np.array([[1], [1], [1]])
cov = np.array([[ 0.5500138 ,  0.07268948, -0.46911148],
                [ 0.07268948,  0.18391358, -0.23438668],
                [-0.46911148, -0.23438668,  0.61144262]])
n = 158

print(check(beta_star, beta, cov, n))

rng = np.random.default_rng(1337)
n = 30
phi = rng.normal(size=(5, 1))
A = rng.normal(size=(5, 5))
sigma = A @ A.T
phi_star = phi + A @ np.array([10/3, 0, 0, 0, 0]).reshape(-1, 1) / np.sqrt(n)

print(check(phi_star, phi, sigma, n))

rng = np.random.default_rng(1337)
n = 30
phi = rng.normal(size=(5, 1))
A = rng.normal(size=(5, 5))
sigma = A @ A.T
phi_star = phi + A @ np.array([10/3 - 0.1, 0, 0, 0, 0]).reshape(-1, 1) / np.sqrt(n)

print(check(phi_star, phi, sigma, n))

False
True
True
